In [5]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np

train_df = pd.read_csv("./split-data/train_new.csv")
test_df = pd.read_csv("./split-data/test_new.csv")
val_df = pd.read_csv("./split-data/val_new.csv")

In [6]:
traingen=ImageDataGenerator(rescale= 1./255)
train_generator=traingen.flow_from_dataframe(
dataframe=train_df,
directory="./dataset",
x_col="img_path",
y_col= "category",
batch_size=128,
shuffle= True,
target_size = (256,256),
class_mode="categorical",
workers=4
)

valgen = ImageDataGenerator(rescale=1./255)
val_generator = valgen.flow_from_dataframe(
    dataframe=val_df,
    directory="./dataset",
    x_col="img_path",
    y_col="category",
    target_size=(256, 256),
    batch_size=128,
    class_mode="categorical",
    workers=4
)

Found 4011 validated image filenames belonging to 46 classes.
Found 3139 validated image filenames belonging to 46 classes.


In [7]:
from keras.models import Model
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.optimizers import SGD,Adam
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from keras.layers import Dense, Flatten
from tensorflow.keras.applications.densenet import DenseNet121

In [ ]:
# Load pretrained DenseNet121 model

# Make classifer deeper
# Add new features (dropout, etc)
# Make more layeres trainable (last 1-2)
base_model = DenseNet121(weights='imagenet', include_top=False, pooling='avg', input_shape=(256, 256, 3))
for i in range(0, len(base_model.layers) - 2):
    base_model.layers[i].trainable = False

# Create new model
base_output = base_model.output
# Add new classifier layers (Dense)
num_classes = 46
layer1 = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(base_output)
dropout1 = Dropout(0.2)(layer1)
layer2 = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(dropout1)
dropout2 = Dropout(0.2)(layer2)
output = Dense(num_classes, activation='softmax')(dropout2)
model = Model(inputs=base_model.inputs, outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
]



In [11]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    callbacks=callbacks
)

Epoch 1/10


2024-12-03 09:44:26.013778: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-12-03 09:44:26.036286: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


32/32 [==============================] - 166s 5s/step - loss: 11.1480 - accuracy: 0.1112 - top_k_categorical_accuracy: 0.3603 - lr: 0.0010
Epoch 2/10
32/32 [==============================] - 177s 6s/step - loss: 5.9359 - accuracy: 0.2536 - top_k_categorical_accuracy: 0.6305 - lr: 0.0010
Epoch 3/10
32/32 [==============================] - 181s 6s/step - loss: 4.3180 - accuracy: 0.3216 - top_k_categorical_accuracy: 0.7056 - lr: 0.0010
Epoch 4/10
32/32 [==============================] - 184s 6s/step - loss: 3.6617 - accuracy: 0.3503 - top_k_categorical_accuracy: 0.7362 - lr: 0.0010
Epoch 5/10
32/32 [==============================] - 189s 6s/step - loss: 3.2684 - accuracy: 0.3782 - top_k_categorical_accuracy: 0.7669 - lr: 0.0010
Epoch 6/10
32/32 [==============================] - 195s 6s/step - loss: 3.0414 - accuracy: 0.3967 - top_k_categorical_accuracy: 0.7699 - lr: 0.0010
Epoch 7/10
32/32 [==============================] - 193s 6s/step - loss: 2.8776 - accuracy: 0.4069 - top_k_categoric

In [12]:
# save the model to a file
model.save("model.h5")

In [13]:
# test the model
testgen = ImageDataGenerator(rescale=1./255)
test_generator = testgen.flow_from_dataframe(
    dataframe=test_df,
    directory="./dataset",
    x_col="img_path",
    y_col="category",
    target_size=(256, 256),
    batch_size=128,
    class_mode="categorical",
    workers=4
)

Found 3133 validated image filenames belonging to 46 classes.


In [ ]:
# test the model on the first batch of test data
model.evaluate(test_generator)

2024-12-03 10:17:14.556660: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


17/25 [===================>..........] - ETA: 46s - loss: 2.8392 - accuracy: 0.3511 - top_k_categorical_accuracy: 0.7477